## Import & config

In [48]:
## utf-8 encoding
import os
import sys
from glob import glob
import requests
import random
import numpy as np
import pandas as pd
import cv2

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from tqdm.notebook import tqdm
from time import time
from torch.optim import *
import timm

import albumentations as A
from albumentations import *
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
timm.list_models()

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_base_patch16_224_in22k',
 'beitv2_large_patch16_224',
 'beitv2_large_patch16_224_in22k',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'coatnet_0_224',
 'coatnet_0_rw_224',
 'coatnet_1_224',
 'coatnet_1_rw_224',
 'coatnet_2_224',
 'coatnet_2_rw_224',
 'coatnet_3_224',
 'coatnet_3_rw_224',
 'coatnet_4_224',
 'coatnet_5_224',
 'coatnet_bn_0_rw_224',
 'coatnet_nano_cc_224',
 'coatnet_nano_rw_224',
 'coatnet_pico_rw_224',
 'coatnet_rmlp_0_rw_224',
 'coatnet_rmlp_1_rw

In [3]:
### Configurations

train_folder_dir = '/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/train'
eval_folder_dir = '/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/eval'

train_imgs_dir = f'{train_folder_dir}/images'
train_labels_path = f'{train_folder_dir}/train.csv'
eval_imgs_dir = f'{eval_folder_dir}/images'
eval_labels_path = f'{eval_folder_dir}/info.csv'

cfg = {
    'model_name' : 'efficientnet_b4',
    'epochs' : 25,
    'lr' : 1e-4,
    'seed':42,
    'num_classes':18
    }

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data Processing

### dataset

In [4]:
df = pd.read_csv(train_labels_path)
df

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59
...,...,...,...,...,...
2695,006954,male,Asian,19,006954_male_Asian_19
2696,006955,male,Asian,19,006955_male_Asian_19
2697,006956,male,Asian,19,006956_male_Asian_19
2698,006957,male,Asian,20,006957_male_Asian_20


In [5]:
## csv → label_codes, img_paths

df = pd.read_csv(train_labels_path)

def get_label(mask_tag, gender, age):
    gender = gender.lower()
    label = 0
    if mask_tag == 'Wear' and gender == 'male' and age<30: # 0:
        label = 0
    elif mask_tag == 'Wear' and gender == 'male' and 30<=age<60: # 1
        label = 1
    elif mask_tag == 'Wear' and gender == 'male' and age>=60: # 2
        label = 2
    elif mask_tag == 'Wear' and gender == 'female' and age<30: # 3
        label = 3
    elif mask_tag == 'Wear' and gender == 'female' and 30<=age<60: # 4
        label = 4
    elif mask_tag == 'Wear' and gender == 'female' and age>=60: # 5
        label = 5
    elif mask_tag == 'Incorrect' and gender == 'male' and age<30: # 6
        label = 6
    elif mask_tag == 'Incorrect' and gender == 'male' and 30<=age<60: # 7
        label = 7
    elif mask_tag == 'Incorrect' and gender == 'male' and age>=60: # 8
        label = 8
    elif mask_tag == 'Incorrect' and gender == 'female' and age<30: # 9
        label = 9
    elif mask_tag == 'Incorrect' and gender == 'female' and 30<=age<60: # 10
        label = 10
    elif mask_tag == 'Incorrect' and gender == 'female' and age>=60: # 11
        label = 11
    elif mask_tag == 'Not Wear' and gender == 'male' and age<30: # 12
        label = 12
    elif mask_tag == 'Not Wear' and gender == 'male' and 30<=age<60: # 13
        label = 13
    elif mask_tag == 'Not Wear' and gender == 'male' and age>=60: # 14
        label = 14
    elif mask_tag == 'Not Wear' and gender == 'female' and age<30: # 15
        label = 15
    elif mask_tag == 'Not Wear' and gender == 'female' and 30<=age<60: # 16
        label = 16
    elif mask_tag == 'Not Wear' and gender == 'female' and age>=60: # 17
        label = 17
    else:
        raise ValueError
    return label

label_codes = []
img_paths = []

for gender, age, folder_name in zip(df['gender'], df['age'], df['path']):
    folder_path = os.path.join(train_imgs_dir, folder_name)
    files = os.listdir(folder_path)
    for file_name in files:
        img_path = os.path.join(folder_path,file_name)
        if 'incorrect' in file_name:
            label = get_label('Incorrect', gender, age)
        elif 'mask' in file_name:
            label = get_label('Wear', gender, age)
        elif 'normal' in file_name:
            label = get_label('Not Wear', gender, age)
        else:
            raise ValueError
        img_paths.append(img_path)
        label_codes.append(label)

In [6]:
for path, code in zip(img_paths, label_codes):
    print(path, code)
    break

/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/train/images/000001_female_Asian_45/mask1.jpg 4


In [7]:
## class distribution
counts = pd.DataFrame(label_codes).value_counts().sort_index()
counts

0     2745
1     2050
2      415
3     3660
4     4085
5      545
6      549
7      410
8       83
9      732
10     817
11     109
12     549
13     410
14      83
15     732
16     817
17     109
dtype: int64

In [8]:
## data split
train_paths, valid_paths, train_labels, valid_labels = train_test_split(img_paths, label_codes,
                                                                        train_size = 0.7,
                                                                        shuffle = True,
                                                                        random_state = cfg['seed'],
                                                                        stratify=label_codes)

In [9]:
train_csv = pd.DataFrame(list(zip(train_paths, train_labels)), columns=['img_path', 'class'])

In [10]:
valid_csv = pd.DataFrame(list(zip(valid_paths, valid_labels)), columns=['img_path', 'class'])

In [11]:
print(len(train_csv), len(valid_csv))

13230 5670


In [12]:
train_csv

,img_path,class
0,/opt/ml/repo/level1_imageclassification_cv-lev...,0
1,/opt/ml/repo/level1_imageclassification_cv-lev...,16
2,/opt/ml/repo/level1_imageclassification_cv-lev...,4
3,/opt/ml/repo/level1_imageclassification_cv-lev...,4
4,/opt/ml/repo/level1_imageclassification_cv-lev...,4
...,...,...
13225,/opt/ml/repo/level1_imageclassification_cv-lev...,4
13226,/opt/ml/repo/level1_imageclassification_cv-lev...,4
13227,/opt/ml/repo/level1_imageclassification_cv-lev...,7
13228,/opt/ml/repo/level1_imageclassification_cv-lev...,3


In [13]:
## input csv, output img, label(class)
class MaskDataSet(Dataset): # train_labels_path
    def __init__(self, path_df, transforms=None):
        self.transforms = transforms
        self.path_df = path_df
        if transforms is None:
            print('If transforms param is None automatically Resize(224), Normalize, ToTensorV2 Apply')
    
    def __getitem__(self, index):
        img_path = self.path_df['img_path'][index]
        img = np.array(Image.open(img_path))
        img_class = self.path_df['class'][index]
        if self.transforms is not None:
            img = self.transforms(image=img)["image"]
        else:
            temp_transforms = A.Compose([A.Resize(height=224, width=224), A.Normalize(), ToTensorV2()])
            img = temp_transforms(image=img)["image"]
        return img, img_class

    def __len__(self):
        return len(self.path_df)

In [14]:
train_transforms = A.Compose([A.Resize(height=224, width=224),
                              A.HorizontalFlip(p=0.5),
                              A.RandomBrightnessContrast(p=0.5),
                              A.GaussianBlur(p=0.5),
                              A.GridDistortion(p=0.5),
                              A.Rotate(limit=30, p=0.5),
                              A.Normalize(mean=(0.56019358,0.52410121,0.501457),
                                          std=(0.23318603,0.24300033,0.24567522)),
                              ToTensorV2()])

valid_transforms = A.Compose([A.Resize(height=224, width=224),
                              A.Normalize(mean=(0.56019358,0.52410121,0.501457),
                                          std=(0.23318603,0.24300033,0.24567522)),
                              ToTensorV2()])

train_dataset = MaskDataSet(train_csv, train_transforms)
valid_dataset = MaskDataSet(valid_csv, valid_transforms)

### dataloader

In [15]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

## Modeling

In [16]:
def get_pretrain_model(config):
    model_name = config['model_name']
    model = timm.create_model(model_name=config['model_name'], pretrained=True, num_classes=config['num_classes'])
    return model

pretrained_model = get_pretrain_model(cfg)
pretrained_model.to(device)

EfficientNet(
  (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
    

In [17]:
## test
batch_images, batch_labels = next(iter(train_loader))
torch.argmax(pretrained_model(batch_images.to(device)), dim=-1)

tensor([ 1, 13, 14,  9,  7,  3,  1,  3,  6,  9,  2, 13,  3, 14,  9,  5,  6, 13,
         9, 12,  9, 14, 14,  1, 14,  5,  3, 14, 14,  9,  4,  1,  3,  0, 17,  5,
         1,  9,  3,  3,  6,  6,  6, 17, 12, 15, 14,  2,  3,  5,  5, 13, 11,  4,
         9,  5, 17,  9,  0, 13,  7, 13, 14,  2], device='cuda:0')

## Training

In [18]:
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optim = AdamW(pretrained_model.parameters(), lr = cfg['lr'])

In [19]:
def train_and_valid(cfg, model, train_loader, valid_loader):
  result = {"train_loss" : [],
            "valid_loss" : [],
            "valid_acc":[],
            "valid_f1":[]}
  for epoch in tqdm((range(1,cfg['epochs']))):
      model.train()
      running_train_loss = []
      running_valid_loss = []

      preds = []
      labels = []


      for iter_idx, (train_imgs, train_labels) in enumerate(train_loader, start=1):
          train_imgs, train_labels = train_imgs.to(device, dtype=torch.float), train_labels.to(device)
          optim.zero_grad()
          train_pred = model(train_imgs)
          train_loss = criterion(train_pred, train_labels)
          train_loss.backward()
          optim.step()
          running_train_loss.append(train_loss.cpu().item())
  
      with torch.no_grad():
        for iter_idx, (valid_imgs, valid_labels) in enumerate(valid_loader, start=1):
          model.eval()
          valid_imgs, valid_labels = valid_imgs.to(device, dtype=torch.float), valid_labels.to(device)
          
          valid_logit = model(valid_imgs)
          valid_loss = criterion(valid_logit, valid_labels)
          valid_preds = valid_logit.argmax(dim=-1)
          
          running_valid_loss.append(valid_loss.cpu().item())
          preds += valid_preds.detach().cpu().numpy().tolist()
          labels += valid_labels.detach().cpu().numpy().tolist()

      train_loss = np.mean(running_train_loss)
      valid_loss = np.mean(running_valid_loss)
      valid_f1 = f1_score(labels, preds, average='macro')
      valid_acc = accuracy_score(labels,preds)
      print(f"{epoch}/{cfg['epochs']} : train_loss:{train_loss:.4f}, \
        valid_loss:{valid_loss:.4f}, valid_acc:{valid_acc*100:.2f}, valid_f1:{valid_f1:.2f}")
      result['train_loss'].append(train_loss)
      result['valid_loss'].append(valid_loss)
      result['valid_acc'].append(valid_acc)
      result['valid_f1'].append(valid_f1)
  return result, model

In [20]:
result, fine_tuned_model = train_and_valid(cfg, pretrained_model, train_loader, valid_loader)

1/25 : train_loss:1.4396,         valid_loss:0.7190, valid_acc:0.77, valid_f1:0.54
2/25 : train_loss:0.5565,         valid_loss:0.4308, valid_acc:0.86, valid_f1:0.70
3/25 : train_loss:0.3621,         valid_loss:0.3477, valid_acc:0.88, valid_f1:0.73
4/25 : train_loss:0.2704,         valid_loss:0.2865, valid_acc:0.90, valid_f1:0.80
5/25 : train_loss:0.2062,         valid_loss:0.2319, valid_acc:0.92, valid_f1:0.83
6/25 : train_loss:0.1640,         valid_loss:0.2052, valid_acc:0.93, valid_f1:0.81
7/25 : train_loss:0.1290,         valid_loss:0.1842, valid_acc:0.94, valid_f1:0.83
8/25 : train_loss:0.1071,         valid_loss:0.1788, valid_acc:0.94, valid_f1:0.86
9/25 : train_loss:0.0755,         valid_loss:0.1494, valid_acc:0.95, valid_f1:0.87
10/25 : train_loss:0.0681,         valid_loss:0.1465, valid_acc:0.95, valid_f1:0.87
11/25 : train_loss:0.0589,         valid_loss:0.1371, valid_acc:0.95, valid_f1:0.87
12/25 : train_loss:0.0497,         valid_loss:0.1160, valid_acc:0.97, valid_f1:0.90
1

## Result check

In [21]:
fine_tuned_model

EfficientNet(
  (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
    

In [22]:
result

{'train_loss': [1.4396341820269967,
  0.5564744252225627,
  0.3620697193675571,
  0.2704026579568927,
  0.20621324001231056,
  0.16396204977867684,
  0.12904674189102247,
  0.1070509271039335,
  0.07546457460663025,
  0.06814021412013234,
  0.05886873244960311,
  0.049698940894436,
  0.03999494971226055,
  0.037379821739384485,
  0.027447926139814446,
  0.030408653178126303,
  0.023942815363440877,
  0.023357978949980172,
  0.02200407462767078,
  0.021968206448383754,
  0.022537287555951042,
  0.016492765378415052,
  0.014309969242237023,
  0.01539718583695802],
 'valid_loss': [0.7189524140250817,
  0.43079741048009207,
  0.3476547695110353,
  0.28648119737927835,
  0.23189921308769268,
  0.20522594602590197,
  0.18423991748707347,
  0.1787889636215869,
  0.14935555373935888,
  0.1464739397712303,
  0.13713322649971488,
  0.11602055690554755,
  0.12756410473434443,
  0.11820126543072669,
  0.13126699581449286,
  0.12159540652810188,
  0.11753330283191347,
  0.11311263364891448,
  0.116

In [23]:
save_file = {'model' : fine_tuned_model.state_dict(),
             'result': result,
             'config': cfg}

In [24]:
save_keyword = cfg['model_name']
save_path = f'/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/checkpoints/2_{save_keyword}.pt'

In [25]:
torch.save(save_file, save_path)

## Inference

In [26]:
eval_imgs_dir = f'{eval_folder_dir}/images'
os.listdir(eval_imgs_dir)

['f3152b9e149732cc7913ad2864bfdba2b820741f.jpg',
 'ccb42a6a1b203887c15edcf7875bd46a678215fc.jpg',
 '935011ab64c0df94353ea6526cad456bdad13cbe.jpg',
 '115d6c044d7206e0a5603f80b342e82e23f3af5f.jpg',
 'f8721a9705ac5f30f9feaf8970cf89674682b8f3.jpg',
 '1938e71f0cb92db5f66c8eeddfef174276ecca2e.jpg',
 '75efc145b8d946b8b6d2a3fab8df8b2f99f94480.jpg',
 'c0fb034bf2b104da56e026d782f3cbff81db1fba.jpg',
 '3a63d338f6b0ffaf2274be9e1844e813e56565c8.jpg',
 'eebe69a972bd1c45181c323b09a074b4e4c9a5e6.jpg',
 '638c3d6b33521052dbd4f16c8e3b25e6d3e4e48d.jpg',
 '60ea7f9a1a01c4089312fc5b2d3255716b0916a0.jpg',
 'f3483594612acb9f02b869b5de6ce8c82a33c205.jpg',
 '754a642ddef7e28bef6f46043348f50ad7b58042.jpg',
 '56036c87a28920b83b4c97a2a48eb12bb3a354b1.jpg',
 'be455126a477f93da9464ddb792ddf343b05123c.jpg',
 '83e383648f08d6e4cc7cf6fff7d9f7b38b800488.jpg',
 '89b723e1c73beee3494ccb2b20fcbdaa7598d464.jpg',
 'ca16a2e22e55d2a0d53370017d44dae93db21f12.jpg',
 'f3d1f43e67bc2a643254b1fe58350077ab2d9e37.jpg',
 'dedd0fe67a8ba05e2d

In [27]:
eval_labels_path = f'{eval_folder_dir}/info.csv'
eval_csv = pd.read_csv(eval_labels_path)

In [28]:
img_path = []
for file_name in eval_csv['ImageID']:
    file_path = os.path.join(eval_imgs_dir, file_name)
    img_path.append(file_path)
    # print(file_path)

In [29]:
test_csv = pd.DataFrame(img_path)
test_csv.columns = ['img_path']
test_csv

,img_path
0,/opt/ml/repo/level1_imageclassification_cv-lev...
1,/opt/ml/repo/level1_imageclassification_cv-lev...
2,/opt/ml/repo/level1_imageclassification_cv-lev...
3,/opt/ml/repo/level1_imageclassification_cv-lev...
4,/opt/ml/repo/level1_imageclassification_cv-lev...
...,...
12595,/opt/ml/repo/level1_imageclassification_cv-lev...
12596,/opt/ml/repo/level1_imageclassification_cv-lev...
12597,/opt/ml/repo/level1_imageclassification_cv-lev...
12598,/opt/ml/repo/level1_imageclassification_cv-lev...


In [30]:
class TestMaskDataSet(Dataset):#train_labels_path
    def __init__(self, path_df, transforms=None):
        self.transforms = transforms
        self.path_df = path_df
        if transforms is None: # transform이 없으면 자동으로 3개의 transform이 적용됨.
            print('If transforms param is None automatically Resize(224), Normalize, ToTensorV2 Apply')
    
    def __getitem__(self, index):
        img_path = self.path_df['img_path'][index]
        img = np.array(Image.open(img_path))
        # img_class = self.path_df['class'][index]
        if self.transforms is not None:
            img = self.transforms(image=img)["image"]
        else:
            temp_transforms = A.Compose([A.Resize(height=224, width=224), A.Normalize(), ToTensorV2()])
            img = temp_transforms(image=img)["image"]
        return img # , img_class

    def __len__(self):
        return len(self.path_df)

In [31]:
test_transforms = A.Compose([A.Resize(height=224, width=224),
                             A.Normalize(mean=(0.56019358,0.52410121,0.501457),
                                         std=(0.23318603,0.24300033,0.24567522)),
                             ToTensorV2()])

In [32]:
test_dataset = TestMaskDataSet(test_csv, transforms = test_transforms)

In [33]:
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [34]:
def inference(model, test_loader):
    total_test_preds = []
    with torch.no_grad():
        for iter_idx, test_imgs in enumerate(test_loader, start=1):
            model.eval()
            test_imgs = test_imgs.to(device, dtype=torch.float)
            test_logit = model(test_imgs)
            test_preds = test_logit.argmax(dim=-1)
            total_test_preds += test_preds.detach().cpu().numpy().tolist()
    return total_test_preds

In [35]:
test_prediction_list = inference(fine_tuned_model, test_loader)

In [36]:
len(test_prediction_list)

12600

In [37]:
df_submission = pd.read_csv('/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/eval/submission.csv')
df_submission['ans']=test_prediction_list

In [38]:
(test_prediction_list == df_submission['ans']).sum()

12600

In [42]:
df_submission

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,14
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,10
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,4


In [46]:
save_path

'/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/submission/2_effifcient.csv'

In [ ]:
/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/submission

In [50]:
submission_file_path = '/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/submission'
save_path = submission_file_path+'/2_effifcient.csv'
df_submission.to_csv(save_path)